# 데이터 전처리 부분

In [1]:
import pandas as pd

In [2]:
columns = ['id', 'text', 'label']

train_data = pd.read_csv('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt', sep='\t', names=columns, skiprows=1).dropna()
test_data = pd.read_csv('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt', sep='\t', names=columns, skiprows=1).dropna()

In [3]:
train_data.to_csv('C:\\Users\\abc\\jupyter\\pytorch\\Sentiment_analysis\\train_data.csv', index=False)
test_data.to_csv('C:\\Users\\abc\\jupyter\\pytorch\\Sentiment_analysis\\test_data.csv', index=False)

In [4]:
from eunjeon import Mecab
mecab = Mecab()

In [5]:
import torch
from torchtext import data

TEXT = data.Field(tokenize = mecab.morphs)
LABEL = data.LabelField(dtype=torch.float)

In [6]:
fields = {'text':('text', TEXT), 'label':('label', LABEL)}

In [8]:
train_data, test_data = data.TabularDataset.splits(
    path = 'C:\\Users\\abc\\jupyter\\pytorch\\Sentiment_analysis',
    train = 'train_data.csv',
    test = 'test_data.csv',
    format = 'csv',
    fields = fields)

In [9]:
SEED = 1234

torch.manual_seed(SEED)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if device == 'cuda':
    torch.cuda.manual_seed_all(SEED)

In [10]:
import random
train_data, valid_data = train_data.split(random_state=random.seed(SEED))

In [11]:
MAX_VOCAB_SIZE = 25000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

In [14]:
print(TEXT.vocab.freqs.most_common(20))

[('.', 112312), ('이', 51529), ('는', 46855), ('영화', 40081), ('다', 38736), ('고', 32805), ('하', 31225), ('도', 23996), ('의', 23642), ('가', 23420), ('은', 21698), ('에', 21482), ('을', 20920), ('한', 17532), ('보', 17463), ('..', 16038), ('게', 15566), (',', 15390), ('들', 15047), ('!', 13782)]


In [16]:
BATCH_SIZE = 64

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    device = device,
    sort_key = lambda x: len(x.text),
    sort_within_batch = False)

In [17]:
next(iter(train_iterator)).text.shape

torch.Size([77, 64])

# 모델 생성 부분

In [18]:
import torch.nn as nn

In [34]:
class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text):
        embedded = self.embedding(text)
        output, hidden = self.rnn(embedded)
        assert torch.equal(output[-1,:,:], hidden.squeeze(0)) # 왜 사용하는지 모르겠음 / 왜 차원을 한개 줄이는지
        
        return self.fc(hidden.squeeze(0))

In [35]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

In [36]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'이 모델은 {count_parameters(model):,} 개의 파라미터를 가지고 있다.')

이 모델은 2,592,105 개의 파라미터를 가지고 있다.


# 모델 훈련, 검증 함수 생성 부분

In [37]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=1e-3)

In [38]:
criterion = nn.BCEWithLogitsLoss()

In [39]:
model = model.to(device)
criterion = criterion.to(device)

In [40]:
def binary_accuracy(preds, y):
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float()
    acc = correct.sum() / len(correct)
    return acc

In [46]:
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    
    for batch in iterator:
        optimizer.zero_grad()
        predictions = model(batch.text).squeeze(1)
        loss = criterion(predictions, batch.label)
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
    
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [51]:
def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.eval()
    
    with torch.no_grad():
        for batch in iterator:
            predictions = model(batch.text).squeeze(1)
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)
            
            epoch_loss += loss.item()
            epoch_acc += acc.item()
            
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [52]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

# 모델 훈련

In [53]:
N_EPOCHS = 5
best_valid_loss = float('inf')

In [55]:
for epoch in range(N_EPOCHS):
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
        
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\t Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:0.2f}%')
    print(f'\t  Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:0.2f}%')

Epoch: 01 | Epoch Time: 0m 11s
	 Train Loss: 0.693 | Train Acc: 50.21%
	  Val. Loss: 0.700 |  Val. Acc: 49.36%
Epoch: 02 | Epoch Time: 0m 11s
	 Train Loss: 0.693 | Train Acc: 50.07%
	  Val. Loss: 0.700 |  Val. Acc: 49.51%
Epoch: 03 | Epoch Time: 0m 11s
	 Train Loss: 0.693 | Train Acc: 50.23%
	  Val. Loss: 0.700 |  Val. Acc: 49.56%
Epoch: 04 | Epoch Time: 0m 11s
	 Train Loss: 0.693 | Train Acc: 50.28%
	  Val. Loss: 0.700 |  Val. Acc: 49.76%
Epoch: 05 | Epoch Time: 0m 11s
	 Train Loss: 0.693 | Train Acc: 50.01%
	  Val. Loss: 0.700 |  Val. Acc: 49.80%


In [57]:
model.load_state_dict(torch.load('tut1-model.pt'))
test_loss, test_acc = evaluate(model, test_iterator, criterion)
print(f'\t Test Loss: {train_loss:.3f} | Test Acc: {train_acc*100:0.2f}%')

	 Test Loss: 0.693 | Test Acc: 50.01%
